#Blurring Images

In this lesson, we will learn how to use `skimage` functions to blur images. When we blur an image, we make the color transition from one side of an edge in the image to another smooth rather than sudden. The effect is to average out rapid changes in pixel intensity. The blur, or smoothing, of an image removes "outlier" pixels that may be noise in the image. Blurring is an example of applying a *low-pass filter* to an image. In computer vision, the term "low-pass filter" applies to removing noise from an image while leaving the majority of the image intact. A blur is a very common operation we need to perform before other tasks such as edge detection. There are several different blurring functions in the `skimage.filters` module, so we will focus on just one here, the Gaussian blur.

##Gaussian Blur

Consider this image of a cat, in particular the area of the image outlined by the white square.

<img src="https://datacarpentry.org/image-processing/fig/05-cat-snap.jpg" alt="large cat image" style="float: left; margin-right:10px;"/>

Now, zoom in on the area of the cat's eye, as shown in the left-hand image below. When we apply a blur filter, we consider each pixel in the image, one at a time. In this example, the pixel we are applying the filter to is highlighted in red, as shown in the right-hand image.

<img src="https://datacarpentry.org/image-processing/fig/05-cat-eye-pixels.jpg" alt="cats-eye-pixels" style="float: left; margin-right:10px;"/>

In a blur, we consider a rectangular group of pixels surrounding the pixel to filter. This group of pixels, called the *kernel*, moves along with the pixel that is being filtered. So that the filter pixel is always in the center of the kernel, the width and height of the kernel must be odd. In the example shown above, the kernel is square, with a dimension of seven pixels.

To apply this filter to the current pixel, a weighted average of the the color values of the pixels in the kernel is calculated. In a Gaussian blur, the pixels nearest the center of the kernel are given more weight than those far away from the center. This averaging is done on a channel-by-channel basis, and the average channel values become the new value for the filtered pixel. Larger kernels have more values factored into the average, and this implies that a larger kernel will blur the image more than a smaller kernel.

To get an idea of how this works, consider this plot of the two-dimensional Gaussian function:

<img src="https://datacarpentry.org/image-processing/fig/05-gaussian-plot.png" alt="Gaussian function" style="float: left; margin-right:10px;"/>

Imagine that plot overlaid over the kernel for the Gaussian blur filter. The height of the plot corresponds to the weight given to the underlying pixel in the kernel. I.e., the pixels close to the center become more important to the filtered pixel color than the pixels close to the edge of the kernel. The shape of the Gaussian function is controlled via its standard deviation, or sigma. A large sigma value results in a flatter shape, while a smaller sigma value results in a more pronounced peak. The mathematics involved in the Gaussian blur filter are not quite that simple, but this explanation gives you the basic idea.

To illustrate the blur process, consider the blue channel color values from the seven-by-seven kernel illustrated above:

```
68  82 71 62 100  98  61 
90  67 74 78  91  85  77 
50  53 78 82  72  95 100 
87  89 83 86 100 116 128 
89 108 86 78  92  75 100 
90  83 89 73  68  29  18 
77 102 70 57  30  30  50
```

The filter is going to determine the new blue channel value for the center pixel – the one that currently has the value 86. The filter calculates a weighted average of all the blue channel values in the kernel, $\left\{76, 83, 81, \ldots, 39, 53, 68\right\}$, giving higher weight to the pixels near the center of the kernel. This weighted average would be the new value for the center pixel. The same process would be used to determine the green and red channel values, and then the kernel would be moved over to apply the filter to the next pixel in the image.

Something different needs to happen for pixels near the edge of the image, since the kernel for the filter may be partially off the image. For example, what happens when the filter is applied to the upper-left pixel of the image? Here are the blue channel pixel values for the upper-left pixel of the cat image, again assuming a seven-by-seven kernel:

```
x   x   x   x   x   x   x
x   x   x   x   x   x   x
x   x   x   x   x   x   x
x   x   x   4   5   9   2 
x   x   x   5   3   6   7 
x   x   x   6   5   7   8 
x   x   x   5   4   5   3 
```

The upper-left pixel is the one with value 4. Since the pixel is at the upper-left corner. there are no pixels underneath much of the kernel; here, this is represented by `x`'s. So, what does the filter do in that situation?

The default mode is to fill in the *nearest* pixel value from the image. For each of the missing `x`'s the image value closest to the `x` is used. If we fill in a few of the missing pixels, you will see how this works:

```
x   x   x   4   x   x   x
x   x   x   4   x   x   x
x   x   x   4   x   x   x
4   4   4   4   5   9   2
x   x   x   5   3   6   7
x   x   x   6   5   7   8
x   x   x   5   4   5   3
```

Another strategy to fill those missing values is to *reflect* the pixels that are in the image to fill in for the pixels that are missing from the kernel.

```
x   x   x   5   x   x   x
x   x   x   6   x   x   x
x   x   x   5   x   x   x
2   9   5   4   5   9   2 
x   x   x   5   3   6   7 
x   x   x   6   5   7   8 
x   x   x   5   4   5   3 
```

A similar process would be used to fill in all of the other missing pixels from the kernel. Other border modes are available; you can learn more about them in the <a href="https://scikit-image.org/docs/dev/user_guide">skimage documentation</a>.

This animation shows how the blur kernel moves along in the original image in order to calculate the color channel values for the blurred image.

<img src="https://datacarpentry.org/image-processing/fig/05-blur-demo.gif" alt="animated blur" style="float: left; margin-right:10px;"/>

`skimage` has built-in functions to perform blurring for us, so we do not have to perform all of these mathematical operations ourselves. The following Python code cells show how to use the skimage Gaussian blur function.

In [ ]:
# one-time imports and set up matplotlib
import skimage.io

from matplotlib import pyplot as plt

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 150

In [ ]:
# read in image
image = skimage.io.imread('https://i.imgur.com/ZAasAJv.png')

# display original image
skimage.io.imshow(image)
plt.show()

In [ ]:
# apply Gaussian blur, creating a new image
import skimage.filters

sigma = 1.0
trunc = 3.5
blurred = skimage.filters.gaussian(image, (sigma, sigma), truncate=trunc, 
                                   multichannel=True)

# display blurred image
skimage.io.imshow(blurred)
plt.show()

The first two parameters to `skimage.filters.gaussian()` are the image to blur, `image`, and a tuple defining the sigma to use in $y$- and $x$-direction, `(sigma, sigma)`. The third parameter, `truncate`, gives the radius of the kernel in terms of sigmas. A Gaussian is defined from -infinity to +infinity. A discrete Gaussian can only approximate the real function. The `truncate` parameter steers at what distance to the center of the function it is not approximated any more. In the above example we set `truncate` to 3.5. With a `sigma` of 1.0 the resulting kernel size would be 7. The default value for truncate in `sklearn` is 4.0. The last parameter is to tell `skimage` how to interpret our image, that has three dimensions, as a multichannel color image. After the blur filter has been executed, the cell wraps things up by displaying the blurred image in a new window.

---
> Re-run the preceding code cells with a sigma of 3.0, and a variety of 
> other sigma values. What can you conclude about the value of sigma 
> and the amount of blurring that takes place?
---

In [ ]:
# TODO: Copy / modify the blurring code, varying the size of the 
# sigma value


---
> Now, modify the cells above so that the kernel supplied to the 
> `gaussian` function uses a rectangular versus square kernel. Then, 
> vary the width and height of the kernel and observe the changes. 
> What have you learned about kernel shape and the amount of blurring
> that takes place?
---

In [ ]:
# TODO: Copy / modify the blurring code, to use a rectangular 
# kernel instead of a square kernel.
# apply Gaussian blur, creating a new image


##Other Methods of Blurring

The Gaussian blur is a way to apply a low-pass filter in skimage. It is often used to remove Gaussian (i. e., random) noise from the image. For other kinds of noise, e.g. "salt and pepper" or "static" noise, a median filter is typically used. See the <a href="https://scikit-image.org/docs/dev/api/skimage.filters.html#module-skimage.filters">`skimage.filter` documentation</a> for a list of available filters.

---
> **Blurring the Bacteria Colony Images**
> 
> As we move further into the workshop, we will see that in order to 
> complete the colony-counting morphometric challenge at the end, we 
> will need to read the bacteria colony images as grayscale, and blur 
> them, before moving on to the tasks of actually counting the 
> colonies. Create Python code cells to read the colony images as 
> grayscale, and then apply a Gaussian blur to each image. 
>
> The colony images can be accessed via these URLs:
> 
> Colony image 1: <a href='https://i.imgur.com/uM0Rt9r.png'>https://i.imgur.com/uM0Rt9r.png</a>
> 
> Colony image 2: <a href='https://i.imgur.com/MAWoq9A.png'>https://i.imgur.com/MAWoq9A.png</a>
> 
> Colony image 3: <a href='https://i.imgur.com/SrG8kTQ.png'>https://i.imgur.com/SrG8kTQ.png</a>
---

In [ ]:
# TODO: write code to read colony images as grayscale, blurring each